# Chapter Summary

<img src="Figures6/S60-Autonomous_Vehicle_with_LIDAR_and_cameras-09.jpg" alt="Splash image with steampunk autonomous car" width="60%" align=center style="vertical-align:middle;margin:10px 0px">


In summary...